In [1]:
from importlib import reload

import datasets
import numpy as np

import src.models.components.feature_extractor_dinov2
import src.models.components.spatiotemporal_encoder

In [2]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [51]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(src_tensor, sampling_rate=25, pad_to_multiple_of=16, return_tensors="pt")

input_values = batched_feature.input_values
attention_mask = batched_feature.attention_mask

print("input_values:", input_values.shape)
print("attention_mask:", attention_mask)

input_values: torch.Size([3, 16, 768])
attention_mask: tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [52]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoderConfig

config = SpatiotemporalEncoderConfig()

model = SpatiotemporalEncoder(config)
encoded = model(input_values, attention_mask)

encoded

Wav2Vec2BaseModelOutput(last_hidden_state=tensor([[[ 0.6495, -0.8987,  0.9071,  ...,  0.6866, -0.9062,  0.1409],
         [ 1.1978, -0.0294,  1.2185,  ...,  0.4299, -1.5511,  0.8242],
         [ 0.9284, -0.4278,  1.3816,  ...,  0.1626, -0.6459,  0.7420],
         ...,
         [ 0.3981,  0.6593, -0.5986,  ...,  0.6849, -1.4441,  1.2985],
         [ 1.4823, -0.6170, -0.9101,  ..., -0.0726, -1.6750,  0.5274],
         [ 2.1547,  0.0249,  0.5231,  ...,  0.3810, -0.0370,  0.2938]],

        [[ 0.5174,  0.1555,  1.6137,  ...,  0.7110, -1.0428,  0.4275],
         [-0.5733, -0.3805, -0.7335,  ...,  1.4192,  0.1213, -0.6701],
         [ 0.6936,  1.6158, -0.4144,  ...,  1.6533, -1.5960,  0.4598],
         ...,
         [ 0.6437,  0.4594,  0.6036,  ...,  0.6496,  0.0118, -0.3845],
         [ 0.1507,  0.5944,  1.0214,  ...,  1.2241, -1.0008, -0.5884],
         [-0.7863,  2.1151,  0.2282,  ...,  0.6408, -2.3383, -1.1621]],

        [[ 0.7854, -1.1146,  0.6117,  ...,  0.9293, -0.7707,  0.8209],
   

In [61]:
reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet


model = SignLanguageNet()

model

SignLanguageNet(
  (encoder): SpatiotemporalEncoder(
    (feature_extractor): SpatialFeatureEncoder()
    (feature_projection): SpatiotemporalFeatureProjection(
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2Encoder(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
        (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,), groups=16)
        (padding): Wav2Vec2SamePadLayer()
        (activation): GELUActivation()
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-5): 6 x Wav2Vec2EncoderLayer(
          (attention): Wav2Vec2Attention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=7

In [67]:
out = model(input_values=input_values, attention_mask=attention_mask, decoder_input_ids=)

out

TypeError: 'int' object is not callable